In [1]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om
    
from openmdao.utils.assert_utils import assert_near_equal

# Driver Recording

A CaseRecorder is commonly attached to the problem’s Driver in order to gain insight into the convergence of the model as the driver finds a solution. By default, a recorder attached to a driver will record the design variables, constraints and objectives.

The driver recorder is capable of capturing any values from any part of the model, not just the design variables, constraints, and objectives.

In [2]:
om.show_options_table("openmdao.core.driver.Driver", recording_options=True)

Option,Default,Acceptable Values,Acceptable Types,Description
excludes,[],N/A,['list'],Patterns for vars to exclude in recording (processed post-includes). Uses fnmatch wildcards
includes,[],N/A,['list'],Patterns for variables to include in recording. Uses fnmatch wildcards
record_constraints,True,"[True, False]",['bool'],Set to True to record constraints at the driver level
record_derivatives,False,"[True, False]",['bool'],Set to True to record derivatives at the driver level
record_desvars,True,"[True, False]",['bool'],Set to True to record design variables at the driver level
record_inputs,True,"[True, False]",['bool'],Set to True to record inputs at the driver level
record_model_metadata,True,"[True, False]",['bool'],Deprecated. Recording of model metadata will always be done
record_objectives,True,"[True, False]",['bool'],Set to True to record objectives at the driver level
record_outputs,True,"[True, False]",['bool'],Set True to record outputs at the driver level.
record_residuals,False,"[True, False]",['bool'],Set True to record residuals at the driver level.


> **_Note:_**  Note that the `excludes` option takes precedence over the `includes` option.

## Driver Recording Example

In the example below, we first run a case while recording at the driver level. Then, we examine the objective, constraint, and design variable values at the last recorded case. Lastly, we print the full contents of the last case, including outputs from the problem that are not design variables, constraints, or objectives. 

Specifically, `y1` and `y2` are some of those intermediate outputs that are recorded due to the use of:

`driver.recording_options['includes'] = ['*']`

In [3]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

import numpy as np

prob = om.Problem(model=SellarDerivatives())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9)
driver.recording_options['includes'] = ['*']
driver.recording_options['record_objectives'] = True
driver.recording_options['record_constraints'] = True
driver.recording_options['record_desvars'] = True
driver.recording_options['record_inputs'] = True
driver.recording_options['record_outputs'] = True
driver.recording_options['record_residuals'] = True

recorder = om.SqliteRecorder("cases.sql")
driver.add_recorder(recorder)

prob.setup()
prob.run_driver()
prob.cleanup()

NL: NLBGS Converged in 8 iterations
NL: NLBGS Converged in 1 iterations
NL: NLBGS Converged in 9 iterations
NL: NLBGS Converged in 10 iterations
NL: NLBGS Converged in 10 iterations
NL: NLBGS Converged in 9 iterations
NL: NLBGS Converged in 6 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: [3.18339395]
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
Optimization Complete
-----------------------------------


In [4]:
cr = om.CaseReader("cases.sql")
driver_cases = cr.list_cases('driver')

driver
rank0:ScipyOptimize_SLSQP|0


driver
rank0:ScipyOptimize_SLSQP|1


driver
rank0:ScipyOptimize_SLSQP|2


driver
rank0:ScipyOptimize_SLSQP|3


driver
rank0:ScipyOptimize_SLSQP|4


driver
rank0:ScipyOptimize_SLSQP|5


driver
rank0:ScipyOptimize_SLSQP|6


In [5]:
assert len(driver_cases) == 7

In [6]:
last_case = cr.get_case(driver_cases[-1])
print(last_case)

driver rank0:ScipyOptimize_SLSQP|6 {'z': array([1.97763888e+00, 5.87897722e-15]), 'x': array([0.]), 'obj': array([3.18339395]), 'y2': array([3.75527777]), 'con1': array([-8.38054071e-11]), 'con2': array([-20.24472223]), 'y1': array([3.16])}


In [7]:
objectives = last_case.get_objectives()
print(objectives)

{'obj': array([3.18339395])}


In [8]:
assert_near_equal(objectives['obj'], 3.18339395, tolerance=1e-8)

5.41351690502717e-10

In [9]:
design_vars = last_case.get_design_vars()
print(design_vars)

{'z': array([1.97763888e+00, 5.87897722e-15]), 'x': array([0.])}


In [10]:
assert_near_equal(design_vars['x'], 0., tolerance=1e-8)
assert_near_equal(design_vars['z'][0], 1.97763888, tolerance=1e-8)

1.7627604631195688e-09

In [11]:
constraints = last_case.get_constraints()
print(constraints)

{'con1': array([-8.38054071e-11]), 'con2': array([-20.24472223])}


In [12]:
assert_near_equal(constraints['con1'], -8.38054071e-11, tolerance=1e-8)
assert_near_equal(constraints['con2'][0], -20.24472223, tolerance=1e-8)

1.4953032152054424e-10

In [13]:
print(last_case.inputs['obj_cmp.x'])

[0.]


In [14]:
assert_near_equal(last_case.inputs['obj_cmp.x'], 0, tolerance=1e-8)

0.0

In [15]:
print(last_case.outputs['z'])

[1.97763888e+00 5.87897722e-15]


In [16]:
assert_near_equal(last_case.outputs['z'][0], 1.97763888, tolerance=1e-8)
assert_near_equal(last_case.outputs['z'][1], 5.87897722e-15, tolerance=1e-8)

2.604323199359837e-10

In [17]:
print(last_case.residuals['obj'])

[3.88027388e-11]


In [18]:
assert_near_equal(last_case.residuals['obj'], 0, tolerance=1e-8)

3.880273879985907e-11

In [19]:
print(last_case['y1'])

[3.16]


In [20]:
assert_near_equal(last_case['y1'], 3.16, tolerance=1e-8)

2.6520698436213565e-11